In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector

In [3]:
from getpass import getpass

In [4]:
password= getpass()

In [5]:
conn = mysql.connector.connect(host="localhost",
                               user="root",
                               password=password)

In [6]:
conn.is_connected()

True

In [7]:
cursor=conn.cursor()

In [8]:



import pandas as pd

def rental_month(conn, month, year):
    cursor = conn.cursor()
    query = f"""SELECT *, YEAR(DATE_FORMAT(sakila.rental.rental_date, "%y%m%d")) AS year FROM sakila.rental;"""
    cursor.execute(query)
    results = cursor.fetchall()
    rentaldf = pd.DataFrame(results, columns=cursor.column_names)
    return rentaldf




In [9]:
month_data = rental_month(conn, "May", 2005)  

In [10]:
month_data
month_data.to_csv("month_data.csv")

In [11]:

conn = mysql.connector.connect(host="localhost",
                               user="root",
                               password=password)

cursor=conn.cursor()


#def rental_count_month(month_data, m, y):
#    if month_data["month_rental"] == m and month_data["year"] == y:
#        customdf=month_data[month_data.groupby(["customer_id"]).agg({"rental_id":"count"})]
#        return customdf
    
def rental_count_month(month_data, m, y):
    bool_mask = month_data[(month_data["month_rental"] == m) & (month_data["year"] == y)]
    customdf = bool_mask.groupby(["customer_id"]).agg({"rental_id":"count"})
    customdf[f"rentals_{m}_{y}"]=bool_mask.groupby(["customer_id"]).agg({"rental_id":"count"})
    return customdf


In [12]:
rental_count_month(month_data,"May",2005)

,rental_id,rentals_May_2005
customer_id,,
1,2,2
2,1,1
3,2,2
5,3,3
6,3,3
...,...,...
594,4,4
595,1,1
596,6,6


In [19]:
def compare_rentals(month_data, m, y, m1, y1):
    df1 = rental_count_month(month_data, m, y)
    df2 = rental_count_month(month_data, m1, y1)

    finaldf = pd.merge(df1, df2, on="customer_id", how="inner")
    
    finaldf["difference"] = df1["rental_id"] - df2["rental_id"]
    
    return finaldf

In [20]:
compare_rentals(month_data,"May",2005,"June",2006)

,rental_id_x,rentals_May_2005,rental_id_y,rentals_June_2006,difference
customer_id,,,,,
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
594,NaN,NaN,NaN,NaN,NaN
595,NaN,NaN,NaN,NaN,NaN
596,NaN,NaN,NaN,NaN,NaN
